### Imports & Paths

In [ ]:
import os
import sys
from pathlib import Path

os.chdir(Path(os.getcwd()).parents[2])
sys.path.append(os.getcwd())

import torch
import numpy as np
from tqdm import tqdm
import plotly.graph_objects as go
from plotly.subplots import make_subplots


### Data Import & Preprocessing

In [ ]:
## Change npz data for different dimensions here ##
dim = "3D"  # 1D, 2D_time, 2D_image, 3D
target = "mth"  # pce, mth

data = np.load("./xai/results/" + target + "_eval_" + dim + "_results.npz")
sensN = data["arr_0"]
ins_abc = data["arr_1"]
del_abc = data["arr_2"]
infid = data["arr_3"]
sens = data["arr_4"]

log_n_max = 2.7
log_n_ticks = 0.4
n_list = np.logspace(0, log_n_max, int(log_n_max / log_n_ticks), base=10.0, dtype=int)

# Changes per Dimension
if target == "pce":
    if dim == "1D":
        ins_abc = ins_abc.clip(None, 1)
        # infid = infid.clip(None,0.008)
        # sens = sens.clip(None,8)
    elif dim == "2D_time":
        pass
        # infid = infid.clip(None,0.0015)
    elif dim == "2D_image":
        ins_abc = ins_abc.clip(None, 0.3)
        del_abc = del_abc.clip(None, 1.0)
        # sens = sens.clip(None,3)
        # infid = infid.clip(None,0.0006)
    elif dim == "3D":
        ins_abc = ins_abc.clip(None, 0.1)
        # infid = infid.clip(None, 0.0001)
else:
    if dim == "1D":
        pass
        # infid = infid.clip(None,50)
        # sens = sens.clip(None,10)
    elif dim == "2D_time":
        ins_abc = ins_abc.clip(None, 1)
        del_abc = del_abc.clip(None, 6)
        # sens = sens.clip(None,6)
        # infid = infid.clip(None,2)
    elif dim == "2D_image":
        ins_abc = ins_abc.clip(None, 25)
        del_abc = del_abc.clip(None, 80)
        # sens = sens.clip(None,3)
        # infid = infid.clip(None,6)


### Evaluation Visualization

In [ ]:
def format_title(title, subtitle=None, subtitle_font_size=14):
    title = f"<b>{title}</b>"
    if not subtitle:
        return title
    subtitle = f'<span style="font-size: {subtitle_font_size}px;">{subtitle}</span>'
    return f"{title}<br>{subtitle}"


if target == "pce":
    title = "Attribution Evaluation PCE"
else:
    title = "Attribution Evaluation Mean Thickness"


fig = make_subplots(rows=1, cols=5)

cd = ["#0059A0", "#5F3893", "#E1462C", "#FF8777"]
methods = ["EG", "IG", "GBP", "GGC"]

for i in range(4):
    fig.add_trace(
        go.Scatter(
            y=np.nanmean(sensN[i], 0),
            x=n_list,
            name=methods[i],
            marker_color=cd[i],
            showlegend=False,
            mode="lines+markers",
        ),
        row=1,
        col=1,
    )

for i in range(4):
    fig.add_trace(
        go.Box(y=ins_abc[i], name=methods[i], marker_color=cd[i], showlegend=False),
        row=1,
        col=2,
    )

for i in range(4):
    fig.add_trace(
        go.Box(y=del_abc[i], name=methods[i], marker_color=cd[i], showlegend=False),
        row=1,
        col=3,
    )

for i in range(4):
    fig.add_trace(
        go.Box(y=sens[i].squeeze(), name=methods[i], marker_color=cd[i]),
        row=1,
        col=4,
    )

for i in range(4):
    fig.add_trace(
        go.Box(y=infid[i].squeeze(), name=methods[i], marker_color=cd[i], showlegend=False),
        row=1,
        col=5,
    )


fig.update_xaxes(tickfont=dict(size=14, family="Helvetica", color="rgb(0,0,0)"))
fig.update_xaxes(
    title="Log N", type="log", range=[-0.1, 3.1], tickvals=[1, 10, 100, 1000], zeroline=False, row=1, col=1
)
fig.update_xaxes(title="Methods", zeroline=True, row=1, col=2)
fig.update_xaxes(zeroline=True, row=1, col=4)

fig.update_yaxes(tickfont=dict(size=14, family="Helvetica", color="rgb(0,0,0)"))
fig.update_yaxes(
    range=[0, 0.305], row=1, col=1
)  # [-0.61, 0.61] [-0.61, 0.61] [-0.01, 0.21] [-0.61, 0.61] [0, 0.81] [-0.21, 0.81] [0, 0.305]
fig.update_yaxes(
    range=[0, 15.1], rangemode="tozero", row=1, col=2
)  # [0,1.01] [0,40.5] [0,0.51] [0,1.02] [0,0.41] [0,25.5] [0,0.105]
fig.update_yaxes(
    range=[0, 201], rangemode="tozero", row=1, col=3
)  # [0,6.05] [0,302] [-0.01,2.01] [-1.1,1.03] [-0.01,6.1] [0,1.02] [0,81] [0,1.04]
fig.update_yaxes(
    range=[-1.1, 1.03],
    rangemode="tozero",
    type="log",
    tickvals=[0.1, 0.2, 0.5, 1, 2, 5, 10],
    zeroline=True,
    row=1,
    col=4,
)  # [-2.1,3.07] [-2.1,3.07] [-1.1,1.03] [-3.1,2.03] [-2.1,2.03] [-4.1,1.03] [-1.1,1.03]
fig.update_yaxes(
    range=[-1.1, 1.03], rangemode="tozero", type="log", tickvals=[0.1, 0.2, 0.5, 1, 2, 5, 10], row=1, col=5
)  # [-8.1,0.1] [-4.1,4.1] [-6.1,-0.9] [-6.1,2.1] [-8.1,0.1] [-6.1,2.1] [-6.1,-2.95]

fig.update_layout(
    legend_title=None,
    legend={"traceorder": "normal"},
    title_y=0.96,
    title_x=0.035,
    template="plotly_white",
    height=400,
    width=1600,
)

fig.write_image("xai/images/" + target + "/" + dim + "/" + dim + "_eval.png", scale=2)

fig.show()
